<a href="https://colab.research.google.com/github/josematez/DeepLearning_SkinCancer/blob/main/Data_Augmentation%2BData_Balance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install deepdish
!pip install numpy==1.15.4

     |████████████████████████████████| 13.9MB 323kB/s 
ERROR: umap-learn 0.4.6 has requirement numpy>=1.17, but you'll have numpy 1.15.4 which is incompatible.
ERROR: tensorflow 2.3.0 has requirement numpy<1.19.0,>=1.16.0, but you'll have numpy 1.15.4 which is incompatible.
ERROR: plotnine 0.6.0 has requirement numpy>=1.16.0, but you'll have numpy 1.15.4 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: astropy 4.1 has requirement numpy>=1.16, but you'll have numpy 1.15.4 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.18.5
    Uninstalling numpy-1.18.5:
      Successfully uninstalled numpy-1.18.5


In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import deepdish as dd


In [2]:
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/ISIC_Dataset')

Mounted at /content/drive


In [3]:
data = dd.io.load('ISIC_dataset.h5')
data_df = pd.DataFrame(data)

In [4]:
X_raw = data_df.image
Y = data_df.cell_type_idx

In [123]:
from imgaug import augmenters as iaa
from imgaug import parameters as iap

"""
Params de entrada: X, Y, num_muestras_clase
Params de salida: X_bal, Y_bal
"""
def balance_and_augmentation_data(X, Y, num_muestras_clase): 

  rows, cols, depth = X[0].shape
  num_clases = Y.nunique()
  num_muestras_init = Y.value_counts()
  print((num_clases*num_muestras_clase, rows, cols, depth))
  X_bal = np.zeros((num_clases*num_muestras_clase, rows, cols, depth), dtype = np.uint8)
  Y_bal = np.zeros((num_clases*num_muestras_clase,), dtype = np.uint8)

  # Se define el operador para data augmentation
  seq = iaa.Sequential([
      iaa.GaussianBlur(
          sigma=iap.Uniform(0.0, 1.0)
      ),
      iaa.Affine(
          rotate=iap.Normal(0.0, 30),
          translate_px=iap.RandomSign(iap.Poisson(3))
      )
  ])

  next_idx = 0

  for cat in range(num_clases):

    n_imgs_generate = num_muestras_clase - num_muestras_init[cat]
    idx_imgs_cat = np.where(Y == cat)[0]

    for idx, img in enumerate(idx_imgs_cat):
      if idx < num_muestras_clase:
        X_bal[next_idx,:,:,:] = X[img]
        Y_bal[next_idx] = Y[img]
        next_idx += 1
      else:
        break

    if n_imgs_generate > 0:
      idx_new_imgs = np.random.randint(0, num_muestras_init[cat], n_imgs_generate)
      for idx in idx_new_imgs:
        X_bal[next_idx,:,:,:] = seq.augment_image(X_raw[idx])
        Y_bal[next_idx] = Y[idx]
        next_idx += 1

  return X_bal, Y_bal 

In [124]:
X_bal, Y_bal = balance_and_augmentation_data(X_raw, Y, 2000)

(14000, 75, 100, 3)


In [130]:
print('X_bal shape: {}'.format(X_bal.shape))
print('Y_bal shape: {}'.format(Y_bal.shape))

X_bal shape: (14000, 75, 100, 3)
Y_bal shape: (14000,)
